In [ ]:
# ============================================================
# 0.2 — Installation des bibliothèques nécessaires
# À exécuter une seule fois au début du notebook (Colab)
# ============================================================

!pip install -q transformers datasets sentence-transformers accelerate faiss-cpu

import torch
import transformers
import datasets
from sentence_transformers import SentenceTransformer

print(" PyTorch version :", torch.__version__)
print(" Transformers version :", transformers.__version__)
print(" Datasets version :", datasets.__version__)

# Vérif rapide du GPU (optionnel, mais motivant pour les étudiants)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(" GPU disponible :", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print(" Pas de GPU détecté, on reste sur CPU.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 35.4 MB/s eta 0:00:00
 PyTorch version : 2.9.0+cu126
 Transformers version : 4.57.1
 Datasets version : 4.0.0
 Pas de GPU détecté, on reste sur CPU.


In [ ]:
# ============================================================
# 1.3 — Première expérience : embeddings & similarité cosinus
# ============================================================

from sentence_transformers import SentenceTransformer
import numpy as np

# 1) Charger le modèle de sentence embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)
print(f" Modèle chargé : {model_name}")

# 2) Définir quelques phrases de test (en français)
sentences = [
    "Ce film était incroyable, j'ai adoré chaque minute.",
    "J'ai vraiment aimé ce film, il était excellent.",
    "Ce film est une perte de temps, je le déteste.",
    "Il fait très beau aujourd'hui, le soleil brille.",
    "Le temps est magnifique, il y a beaucoup de soleil.",
    "Je dois acheter du pain à la boulangerie."
]

print("\n Phrases test :")
for i, s in enumerate(sentences):
    print(f"{i}: {s}")

# 3) Calculer les embeddings (matrice [nb_phrases, dim_embedding])
embeddings = embedder.encode(sentences, convert_to_numpy=True, normalize_embeddings=True)
print("\n Forme de la matrice d'embeddings :", embeddings.shape)

# 4) Fonction de similarité cosinus
def cosine_similarity_matrix(X):
    """
    X : matrice (n, d) où n = nb de phrases, d = dimension embedding
    Retourne une matrice (n, n) des similarités cosinus.
    """
    # X est déjà normalisé (norme 1) => produit scalaire = cosinus
    return np.matmul(X, X.T)

sim_matrix = cosine_similarity_matrix(embeddings)

# 5) Afficher la matrice de similarité de manière lisible
np.set_printoptions(precision=2, suppress=True)
print("\n Matrice de similarité cosinus entre phrases :\n")
print(sim_matrix)

# 6) Afficher quelques similarités ciblées
def show_similarity(i, j):
    print(f"\nSim({i}, {j}) = cosinus( phrase[{i}], phrase[{j}] ) = {sim_matrix[i, j]:.3f}")
    print(f"  phrase[{i}] : {sentences[i]}")
    print(f"  phrase[{j}] : {sentences[j]}")

# Phrases très proches en sens (film positif)
show_similarity(0, 1)

# Film positif vs film négatif
show_similarity(0, 2)

# Deux phrases sur la météo
show_similarity(3, 4)

# Phrase sur la météo vs phrase sur la boulangerie
show_similarity(3, 5)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Modèle chargé : sentence-transformers/all-MiniLM-L6-v2

 Phrases test :
0: Ce film était incroyable, j'ai adoré chaque minute.
1: J'ai vraiment aimé ce film, il était excellent.
2: Ce film est une perte de temps, je le déteste.
3: Il fait très beau aujourd'hui, le soleil brille.
4: Le temps est magnifique, il y a beaucoup de soleil.
5: Je dois acheter du pain à la boulangerie.

 Forme de la matrice d'embeddings : (6, 384)

 Matrice de similarité cosinus entre phrases :

[[1.   0.72 0.6  0.48 0.47 0.37]
 [0.72 1.   0.54 0.51 0.49 0.51]
 [0.6  0.54 1.   0.45 0.57 0.52]
 [0.48 0.51 0.45 1.   0.64 0.51]
 [0.47 0.49 0.57 0.64 1.   0.47]
 [0.37 0.51 0.52 0.51 0.47 1.  ]]

Sim(0, 1) = cosinus( phrase[0], phrase[1] ) = 0.719
  phrase[0] : Ce film était incroyable, j'ai adoré chaque minute.
  phrase[1] : J'ai vraiment aimé ce film, il était excellent.

Sim(0, 2) = cosinus( phrase[0], phrase[2] ) = 0.599
  phrase[0] : Ce film était incroyable, j'ai adoré chaque minute.
  phrase[2] : Ce film est

In [ ]:
documents = [
    """La cybersécurité regroupe l’ensemble des techniques et
    outils destinés à protéger les systèmes informatiques
    contre les attaques et les accès non autorisés.""",

    """Les pare-feux modernes utilisent des règles dynamiques
    et des modèles comportementaux pour bloquer le trafic
    suspect en temps réel.""",

    """La cryptographie assure la confidentialité et
    l’intégrité des données grâce à des algorithmes de
    chiffrement et de signature.""",

    """Pour détecter une intrusion, on collecte des journaux
    d’événements, puis on entraîne un modèle qui repère les
    anomalies dans ces données.""",

    """Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce à un condensat unique créé
    par une fonction de hachage.""",

    """Le Zero Trust repose sur l’idée qu’aucune entité ne
    doit être automatiquement considérée comme fiable, même
    à l’intérieur du réseau.""",

    """Un système de détection avancée peut combiner analyse
    comportementale et intelligence artificielle pour
    identifier des menaces nouvelles ou évolutives."""
]


In [ ]:
# ============================================================
# 2.2 — Encoder les documents
# ============================================================

# On réutilise le modèle embedder chargé dans S1
doc_embeddings = embedder.encode(
    documents,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embeddings documents :", doc_embeddings.shape)


Embeddings documents : (7, 384)


In [ ]:
# ============================================================
# 2.3 — Fonction de recherche sémantique
# ============================================================

def search(query, top_k=3):
    """
    Retourne les top_k documents les plus proches de la requête.
    """
    # Encoder la requête
    query_emb = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]

    # Calcul des similarités cosinus (produit scalaire)
    similarities = np.dot(doc_embeddings, query_emb)

    # Indices des meilleurs passages
    best_idx = np.argsort(similarities)[::-1][:top_k]

    results = []
    for idx in best_idx:
        results.append({
            "document_index": idx,
            "score": float(similarities[idx]),
            "content": documents[idx].strip()
        })
    return results

print(" Fonction de recherche sémantique prête.")

 Fonction de recherche sémantique prête.


In [ ]:
# ============================================================
# 2.4 — Tests du moteur de recherche sémantique
# ============================================================

queries = [
    "C'est quoi le NLP ?",
    "Comment fonctionne un Transformer ?",
    "Comment entraîner un modèle de classification ?",
    "Explique le RAG.",
    "Qu'est-ce qu'un chatbot ?"
]

for q in queries:
    print("\n==============================")
    print(" Requête :", q)
    results = search(q, top_k=2)
    for r in results:
        print(f"\n Score : {r['score']:.3f}")
        print(f" Passage : {r['content']}")



 Requête : C'est quoi le NLP ?

 Score : 0.409
 Passage : Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce à un condensat unique créé
    par une fonction de hachage.

 Score : 0.374
 Passage : Le Zero Trust repose sur l’idée qu’aucune entité ne
    doit être automatiquement considérée comme fiable, même
    à l’intérieur du réseau.

 Requête : Comment fonctionne un Transformer ?

 Score : 0.379
 Passage : Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce à un condensat unique créé
    par une fonction de hachage.

 Score : 0.267
 Passage : Le Zero Trust repose sur l’idée qu’aucune entité ne
    doit être automatiquement considérée comme fiable, même
    à l’intérieur du réseau.

 Requête : Comment entraîner un modèle de classification ?

 Score : 0.436
 Passage : Pour détecter une intrusion, on collecte des journaux
    d’événements, puis on entraîne un modèle qui repère les
    anomalies dans ces données.

 Score

In [ ]:
# ============================================================
# 2.4 — Tests du moteur de recherche sémantique
# ============================================================

queries = [
    "C'est quoi le NLP ?",
    "Comment fonctionne un Transformer ?",
    "Comment entraîner un modèle de classification ?",
    "Explique le RAG.",
    "Qu'est-ce qu'un chatbot ?"
]

for q in queries:
    print("\n==============================")
    print(" Requête :", q)
    results = search(q, top_k=2)
    for r in results:
        print(f"\n Score : {r['score']:.3f}")
        print(f" Passage : {r['content']}")



 Requête : C'est quoi le NLP ?

 Score : 0.409
 Passage : Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce à un condensat unique créé
    par une fonction de hachage.

 Score : 0.374
 Passage : Le Zero Trust repose sur l’idée qu’aucune entité ne
    doit être automatiquement considérée comme fiable, même
    à l’intérieur du réseau.

 Requête : Comment fonctionne un Transformer ?

 Score : 0.379
 Passage : Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce à un condensat unique créé
    par une fonction de hachage.

 Score : 0.267
 Passage : Le Zero Trust repose sur l’idée qu’aucune entité ne
    doit être automatiquement considérée comme fiable, même
    à l’intérieur du réseau.

 Requête : Comment entraîner un modèle de classification ?

 Score : 0.436
 Passage : Pour détecter une intrusion, on collecte des journaux
    d’événements, puis on entraîne un modèle qui repère les
    anomalies dans ces données.

 Score

In [ ]:
# ============================================================
# 3.1 — Chargement d'un modèle génératif (Flan-T5)
# ============================================================

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

gen_model_name = "google/flan-t5-small"  # on peut passer à flan-t5-base plus tard
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

gen_model = gen_model.to(device)

print(f" Modèle génératif chargé : {gen_model_name}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

 Modèle génératif chargé : google/flan-t5-small


In [ ]:
# ============================================================
# 3.2 — Construction du prompt RAG
# ============================================================

def build_rag_prompt(question, retrieved_passages):
    """
    Construit un prompt texte pour le modèle génératif,
    en incluant les documents récupérés.
    """
    context = ""
    for i, passage in enumerate(retrieved_passages):
        context += f"[DOC {i+1}] {passage['content']}\n"

    prompt = (
        "Tu es un assistant qui répond uniquement à partir des documents fournis.\n"
        "Si l'information n'est pas présente dans les documents, dis que tu ne sais pas.\n\n"
        "Documents :\n"
        f"{context}\n"
        "Question : "
        f"{question}\n\n"
        "Réponse en français clair et pédagogique :"
    )
    return prompt

In [ ]:
# ============================================================
# 3.3 — Fonction de Question-Réponse RAG
# ============================================================

def rag_answer(question, top_k=3, max_new_tokens=128):
    """
    1. Retrieve des passages pertinents à partir des documents.
    2. Construit un prompt incluant ces passages.
    3. Utilise un modèle génératif (Flan-T5) pour produire une réponse.
    4. Retourne (passages_utilisés, réponse).
    """
    # 1) Récupération des passages
    retrieved = search(question, top_k=top_k)

    # 2) Construction du prompt
    prompt = build_rag_prompt(question, retrieved)

    # 3) Génération
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    output = gen_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False  # greedy pour la pédagogie (comportement déterministe)
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    return retrieved, answer

In [ ]:
# ============================================================
# 3.4 — Tests du mini système RAG
# ============================================================

test_questions = [
    "Explique ce qu'est le NLP.",
    "Comment fonctionne un modèle Transformer ?",
    "C'est quoi un embedding de phrase ?",
    "Qu'est-ce que le RAG en intelligence artificielle ?",
    "Comment entraîne-t-on un modèle de classification de texte ?",
    "Qu'est-ce qu'un chatbot intelligent dans ce contexte ?"
]

for q in test_questions:
    print("\n" + "="*70)
    print("❓ Question :", q)

    passages, answer = rag_answer(q, top_k=3)

    print("\n Passages utilisés :")
    for i, p in enumerate(passages):
        print(f"\n[DOC {i+1}] (score={p['score']:.3f})")
        print(p["content"])

    print("\n Réponse RAG :")
    print(answer)


❓ Question : Explique ce qu'est le NLP.

 Passages utilisés :

[DOC 1] (score=0.455)
Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce à un condensat unique créé
    par une fonction de hachage.

[DOC 2] (score=0.408)
Le Zero Trust repose sur l’idée qu’aucune entité ne
    doit être automatiquement considérée comme fiable, même
    à l’intérieur du réseau.

[DOC 3] (score=0.323)
La cryptographie assure la confidentialité et
    l’intégrité des données grâce à des algorithmes de
    chiffrement et de signature.

 Réponse RAG :
Explique ce que l’LP. Explique l’attention en français claire et l’enseignement :

❓ Question : Comment fonctionne un modèle Transformer ?

 Passages utilisés :

[DOC 1] (score=0.369)
Les pare-feux modernes utilisent des règles dynamiques
    et des modèles comportementaux pour bloquer le trafic
    suspect en temps réel.

[DOC 2] (score=0.346)
Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce 

In [ ]:
# ============================================================
# 4.2 — Construction du prompt conversationnel
# ============================================================

def build_chat_prompt(history, question, retrieved_passages, max_history=3):
    """
    Construit un prompt incluant :
    - un historique (dernier max_history tours),
    - les documents pertinents,
    - la question actuelle.
    """

    # 1) Historique compact (fenêtre glissante)
    hist_text = ""
    for (q, a) in history[-max_history:]:
        hist_text += f"Utilisateur : {q}\nAssistant : {a}\n"

    # 2) Documents retrouvés
    docs_text = ""
    for i, doc in enumerate(retrieved_passages):
        docs_text += f"[DOC {i+1}] {doc['content']}\n"

    # 3) Construction finale du prompt
    prompt = (
        "Tu es un assistant pédagogique qui répond uniquement à partir des documents fournis.\n"
        "Si l'information n'est pas dans les documents, dis que tu ne sais pas.\n\n"
        f"Historique (contexte conversationnel) :\n{hist_text}\n"
        f"Documents pertinents :\n{docs_text}\n"
        f"Question de l'utilisateur : {question}\n\n"
        "Réponse en français :"
    )
    return prompt

In [ ]:
# ============================================================
# 4.3 — Fonction de Chatbot RAG complet
# ============================================================

def chatbot_rag_turn(history, question, top_k=3, max_new_tokens=128):
    """
    Effectue un tour de conversation :
    - retrieve
    - prompt
    - génération
    - mise à jour de l'historique
    """

    # 1) Retrieval
    retrieved = search(question, top_k=top_k)

    # 2) Prompt complet (historique + docs)
    prompt = build_chat_prompt(history, question, retrieved)

    # 3) Génération
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    output = gen_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    # 4) Mise à jour de l'historique
    history.append((question, answer))

    return answer, retrieved, history

In [ ]:
# ============================================================
# 4.4 — Boucle interactive du chatbot RAG
# ============================================================

def chat():
    print("🤖 Chatbot RAG (tape 'quit' ou 'exit' pour arrêter)\n")
    history = []

    while True:
        user_input = input("Vous : ")
        if user_input.lower() in ["quit", "exit", "stop"]:
            print("Assistant : Au revoir ! 👋")
            break

        answer, retrieved, history = chatbot_rag_turn(history, user_input)

        print("\nAssistant :", answer)
        print("\n📚 Passages utilisés :")
        for i, p in enumerate(retrieved):
            print(f"\n[DOC {i+1}] (score={p['score']:.3f})")
            print(p["content"])
        print("\n" + "-"*70)

# chat()  # Décommenter pour tester en local ou dans un terminal


In [ ]:
# ============================================================
# 4.A — Interface simple : RAG "one-shot" (sans historique)
# ============================================================

def ask_rag(question, top_k=3):
    """
    Pose une question au système RAG (sans historique de chat).
    Affiche directement la réponse et les passages utilisés.
    """
    passages, answer = rag_answer(question, top_k=top_k)

    print("❓ Question :", question)
    print("\n💬 Réponse RAG :")
    print(answer)

    print("\n📚 Passages utilisés :")
    for i, p in enumerate(passages):
        print(f"\n[DOC {i+1}] (score={p['score']:.3f})")
        print(p["content"])
    print("\n" + "-"*70)


In [ ]:
# ============================================================
# Exemple de test "one-shot"
# ============================================================

query = "Explique le RAG."

print("\n==============================")
print(" Requête :", query)

results = search(query, top_k=2)

for r in results:
    print(f"\n Score : {r['score']:.3f}")
    print(f" Passage : {r['content']}")



 Requête : Explique le RAG.

 Score : 0.346
 Passage : Le Zero Trust repose sur l’idée qu’aucune entité ne
    doit être automatiquement considérée comme fiable, même
    à l’intérieur du réseau.

 Score : 0.345
 Passage : Les empreintes numériques permettent d’identifier un
    fichier ou un message grâce à un condensat unique créé
    par une fonction de hachage.
